In [3]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
print(torch.__version__)

2.4.0+cu118


In [4]:
from huggingface_hub import notebook_login

notebook_login()

## SQL db 
 create a database with sqlalchemy

In [5]:
from sqlalchemy import (
    create_engine, # creates a new database
    MetaData,
    Table,
    Column,
    String,
    Integer,
    Float,
    insert,
    inspect,
    text,
)

engine = create_engine("sqlite:///:memory:") # sqlite is SQL db engien written in C
metadata_obj = MetaData() #The MetaData object contains all of the schema constructs we’ve associated with it. 


# create city SQL table
table_name = "receipts"
# create a table that takes table name and metadata obj.
# then define the columns in this table with their datatypes
# specify if they are primary keys or not, i.e. are they unique?

receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("customer_name", String(16), primary_key=True),
    Column("price", Float),
    Column("tip", Float),
)
metadata_obj.create_all(engine)



# fill in the data
rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
]
for row in rows:
    # *list means treate the elements in list as positional args to function
    # **dict means treat the elements in the dict as named or wword args to function
    stmt = insert(receipts).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [6]:
## check the DB
with engine.connect() as con:
    rows = con.execute(text("""SELECT * from receipts"""))
    for row in rows:
        print(row)

(1, 'Alan Payne', 12.06, 1.2)
(2, 'Alex Mason', 23.86, 0.24)
(3, 'Woodrow Wilson', 53.43, 5.43)
(4, 'Margaret James', 21.11, 1.0)


## building the agent to get SQL queries

In [7]:
# the table information from SQL db
inspector = inspect(engine)
columns_info = [(col["name"], col["type"]) for col in inspector.get_columns("receipts")]

table_description = "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
print(table_description)


Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT


### SQL agent tool will need some mandatory arguments as 
 - the description of table that we retrived with above code
 - a name for the agent /system/ tool
 - diction of inputs to the too. what inputs should the agent expect ? type and its description
 - an output type for UI compatibility later on
 - a forward method that gets called when tool is called

### agentic tool

An agent is a system that uses an LLM as its engine, and it has access to functions called tools.
These tools are functions for performing a task, and they contain all necessary description for the agent to properly use them.

In [8]:
from transformers.agents import Tool # import agentic tool framework


# create the tool

class SQLExecutorTool(Tool):
    name = "sql_engine"
    description = f"""allows you to perform SQL querie son the table. Returns a string representation of the result.
    The table is named {table_name}. The description of this table is as follows: \n {table_description}"""
    inputs = {
        "query":{
            "type":"text",
            "description":f"The query to perform. This should be correct SQL."
        }
    }
    output_type = "text"

    def forward(self, query: str) -> str:
        output = ""
        with engine.connect() as con:
            rows = con.execute(text(query))
            for row in rows:
                output += "\n" + str(row)

        return output

In [9]:
# reactcode agent that operates according to the ReAct framework. 
# it has a inbuilt feedback loop

In [9]:
from transformers.agents import ReactCodeAgent, HfEngine # to call LLM we need LLM engine

agent = ReactCodeAgent(
    tools = [SQLExecutorTool], # CURRENTLY we have just one tool
    llm_engine=HfEngine("hugging-quants/Meta-Llama-3.1-8B-Instruct-GPTQ-INT4") # plug any LLM model from HF. I have a quantised version here
)

AttributeError: type object 'SQLExecutorTool' has no attribute 'repo_id'

In [10]:
agent.run("Can you give me the name of the client who got the most expensive receipt?")


NameError: name 'agent' is not defined

In [2]:
from transformers.agents import CodeAgent, ReactCodeAgent, HfEngine
llm_engine=HfEngine("hugging-quants/Meta-Llama-3.1-8B-Instruct-GPTQ-INT4")
# agent = ReactCodeAgent(tools=[], llm_engine=llm_engine, add_base_tools=True)
agent = CodeAgent(tools=[], add_base_tools=True)

agent.run("Could you translate this sentence from French, say it out loud and give me the audio.",
    sentence="Où est la boulangerie la plus proche?",)

======== New task ========
Could you translate this sentence from French, say it out loud and give me the audio.
You have been provided with these initial arguments: {'sentence': 'Où est la boulangerie la plus proche?'}.
==== Agent is executing the code below:
original_sentence = {'sentence': 'Où est la boulangerie la plus proche?'}
translated_sentence = translator(text=original_sentence['sentence'], src_lang="French", tgt_lang="English")
print(f"The translated sentence is {translated_sentence}.")
audio_sentence = text_to_speech(text=translated_sentence)
final_answer(audio_sentence)
====


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--facebook--nllb-200-distilled-600M. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Error in execution: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 2.00 GiB of which 0 bytes is free. Of the allocated memory 1.72 GiB is allocated by PyTorch, and 18.35 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables). Be sure to provide correct code.
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\agents\agents.py", line 627, in run
    output = self.python_evaluator(
             ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\agents\python_interpreter.py", line 893, in evaluate_python_code
    result = evaluate_ast(node, state, static_tools, custom_tools, authorized_imp

'Error in execution: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 2.00 GiB of which 0 bytes is free. Of the allocated memory 1.72 GiB is allocated by PyTorch, and 18.35 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables). Be sure to provide correct code.'